In [ ]:
import pickle
from pathlib import Path
from typing import Dict

import networkx as nx
import pandas as pd
from dowhy import CausalModel

from structure_data import choose_columns, preprocess_data

ROOT = Path('./')
RAW_DATA = ROOT / "raw_data/csv/eqls_2007and2011.csv"
DICT_PATH = ROOT / "data/dictionary.json"
GRAPH_PATH = ROOT / "graphs/full_causal.gpickle"
TREATMENT = "Y11_Q57"
OUTCOME = "Y11_MWIndex"


def load_data() -> pd.DataFrame:
    """Load and preprocess the raw EQLS data."""
    df = choose_columns()
    df = preprocess_data(
        df,
        na_threshold=0.5,
        impute_strategy="drop",
        treatment_dichotomize_value="median",
        treatment_column=TREATMENT,
    )
    return df


def load_graph() -> nx.DiGraph:
    """Load the causal graph describing relationships among variables."""
    with open(GRAPH_PATH, "rb") as f:
        return pickle.load(f)


def estimate_effects(df: pd.DataFrame, graph: nx.DiGraph) -> Dict[str, float]:
    """Estimate ATE using several DoWhy estimators."""
    model = CausalModel(
        data=df,
        treatment=TREATMENT,
        outcome=OUTCOME,
        graph=nx.nx_pydot.to_pydot(graph).to_string(),
    )
    estimand = model.identify_effect()
    methods = [
        "backdoor.propensity_score_matching",
        "backdoor.propensity_score_weighting",
        "backdoor.propensity_score_stratification",
        "backdoor.linear_regression",
        "backdoor.distance_matching",
    ]
    kwargs = {
        "backdoor.distance_matching": {
            "method_params": {'distance_metric': "minkowski", 'p': 2}
        }
    }
    results: Dict[str, float] = {}
    for m in methods:
        try:
            est = model.estimate_effect(estimand, method_name=m, **kwargs.get(m, {}))
            results[m] = float(est.value)
        except Exception as e:
            print(f"[!] Estimation with {m} failed: {e}")
            results[m] = float("nan")
    return results

In [ ]:
df = load_data()
graph = load_graph()
results = estimate_effects(df, graph)
print("Estimation results (ATE):")
for name, val in results.items():
    print(f"  {name:<40} {val:.4f}")